<a href="https://colab.research.google.com/github/amazon4b/Learning-Machine/blob/learner1/%EA%B2%BD%EA%B8%B0%EB%B6%84%EC%84%9D%2Ck_means(k%3D4)%2Cmeanshift%2C_minmax%EC%8A%A4%EC%BC%80%EC%9D%BC%EB%A7%81_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

# 경고 메시지를 출력하지 않도록 설정
warnings.filterwarnings("ignore")

# 경고 메시지를 발생시키는 함수
def example_function():
    warnings.warn("이 함수는 예제일 뿐입니다.", UserWarning)

# 함수 호출
example_function()

In [2]:
# 기본패키지 설치
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.cluster import KMeans # 케이민즈 알고리즘
from sklearn.preprocessing import StandardScaler# 스텐다드 스케일러
# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc

In [3]:
# 데이터프레임 불러오기
economics = pd.read_csv('./datasets/resampled_data_final.csv',encoding="euc-kr")
economics['DATE'] = pd.to_datetime(economics['DATE'])
economics['DATE'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 289 entries, 0 to 288
Series name: DATE
Non-Null Count  Dtype         
--------------  -----         
287 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.4 KB


In [4]:
economics.drop(['장단기10년물국채/3개월금리차','신규실업수당청구','금융상태지수레버리지','제조업신규주문'],axis=1,inplace=True)

In [5]:
economics[economics.isna().any(axis=1)] # 결측치 확인

,DATE,장단기10년물국채/2년물금리차,국제유가,실업률,신규주택착공건수,주택가격지수,금융상태지수,소비자감성지수,기준금리,M2통화량,경기동행종합,산업생산지수,소매판매
287,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
economics = economics.iloc[:287 , ] # 결측치 제거
economics[economics.isna().any(axis=1)] # 결측치 확인

,DATE,장단기10년물국채/2년물금리차,국제유가,실업률,신규주택착공건수,주택가격지수,금융상태지수,소비자감성지수,기준금리,M2통화량,경기동행종합,산업생산지수,소매판매


In [7]:
# 2000년~2019년만 군집화하기 위함 , 코로나 데이터가 이상치로 잡히기 때문에 군집이 제대로 형성되지 않음
economics = economics.loc[economics['DATE'].dt.year <=2019 ]

In [8]:
# 주택가격지수. 제조업신규주문 , 경기동행종합지수는 차분하고 (변동치로 변경한다는 뜻)
# 제조업 신규주문은 앞으로 어떻게 처리할지 일단 보류
#M2와 소매판매는 변화율로 바꾸는 전처리 시행
economics.주택가격지수 = economics.주택가격지수.diff().fillna(method= 'bfill')
economics.경기동행종합 = economics.경기동행종합.diff().fillna(method= 'bfill')
economics['M2차분'] = economics.M2통화량.diff().fillna(method= 'bfill')
economics['M2변화율'] = economics.M2차분/ (economics.M2통화량.shift(1)) * 100
#economics['M2변화율'] .fillna(method = 'ffill')
economics['소매판매차분'] = economics.소매판매.diff().fillna(method='bfill')
economics['소매판매변화율'] = economics.소매판매차분/ (economics.소매판매.shift(1)) * 100


# 차분, 변화율 확인
economics[['M2통화량','M2차분','M2변화율','소매판매','소매판매차분','소매판매변화율']].head(10)
# 결측치를 다음날 데이터로 채워주기
economics['M2변화율'] = economics['M2변화율'].fillna(method = 'bfill')
economics['소매판매변화율'] = economics['소매판매변화율'].fillna(method = 'bfill')
# M2, 소매판매 원데이터를 지웠습니다.
economics.drop(['M2통화량','M2차분','소매판매','소매판매차분'],axis = 1,inplace = True)

In [9]:
# 다시 날짜를 인덱스로 설정해주기
economics = economics.set_index('DATE')

#### 스케일링

In [10]:
# 민멕스 스케일링
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(economics)

# 데이터 변환
mMscaled_data = mMscaler.transform(economics)

# 데이터 프레임으로 저장
mMscaled_data = pd.DataFrame(data = mMscaled_data, index = economics.index , columns = economics.columns)

#### GaussianMixture 군집화 비교분석

In [11]:
pca_df0 = mMscaled_data.copy()

In [12]:
from sklearn.decomposition import PCA

pca = PCA()

pca_df = pd.DataFrame(pca.fit_transform(pca_df0))

# 주성분들이 설명하는 분산의 비율을 확인하는 코드
# Access explained variance ratios
explained_variance_ratios = pca.explained_variance_ratio_

# Print or use the explained variance ratios
print("Explained Variance Ratios:")
print(explained_variance_ratios)

Explained Variance Ratios:
[0.47486627 0.18528129 0.13890905 0.05980781 0.03945728 0.02936114
 0.02662438 0.01594366 0.01311452 0.00692286 0.00604826 0.00366349]


In [13]:
# components 7개로 설정
pca_df = pca_df.drop(columns = [7,8,9,10,11])
pca_df.head()

,0,1,2,3,4,5,6
0,-0.822012,0.369716,0.161866,-0.054982,-0.302402,-0.056911,-0.095982
1,-0.873760,0.363113,0.193319,-0.104235,-0.280930,-0.054163,-0.085926
2,-0.909069,0.247894,0.217418,-0.114760,-0.329716,0.145272,-0.117530
3,-0.949386,0.282315,0.267522,0.006774,-0.246777,0.352583,-0.084640
4,-0.934304,0.294915,0.259300,-0.130338,-0.403937,-0.162235,0.067947


In [14]:
# PCA한 데이터 스케일링
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(pca_df)

# 데이터 변환
scaled_pca = mMscaler.transform(pca_df)

# 데이터 프레임으로 저장
scaled_pca = pd.DataFrame(data = scaled_pca, index = pca_df.index , columns = pca_df.columns)

In [15]:
scaled_pca

,0,1,2,3,4,5,6
0,0.065668,0.753955,0.475056,0.256606,0.125217,0.365264,0.320348
1,0.038989,0.748406,0.497495,0.213235,0.151697,0.368174,0.334068
2,0.020786,0.651579,0.514689,0.203968,0.091533,0.579361,0.290950
3,0.000000,0.680505,0.550434,0.310986,0.193816,0.798888,0.335821
4,0.007775,0.691094,0.544568,0.190250,0.000000,0.253733,0.543999
...,...,...,...,...,...,...,...
235,0.236399,0.170451,0.397957,0.449650,0.494207,0.400489,0.522970
236,0.235087,0.128080,0.378530,0.479030,0.466752,0.476706,0.414215
237,0.254693,0.217826,0.385363,0.529593,0.500461,0.516848,0.428131
238,0.255461,0.172713,0.342116,0.508544,0.475757,0.436842,0.429567


In [16]:
from sklearn.mixture import GaussianMixture
from numpy import unique
from numpy import where

# define the model : n_components=3 (cluster 갯수)
gmm = GaussianMixture(n_components=5, random_state=24)

# train the model
gmm_label = gmm.fit(scaled_pca).predict(scaled_pca)

# assign each data point to a cluster
scaled_pca['gmm_cluster'] = gmm_label

# get all of the unique clusters
gaussian_clusters = unique(scaled_pca['gmm_cluster'])
print(gaussian_clusters)

scaled_pca['gmm_cluster'].value_counts()

[0 1 2 3 4]


gmm_cluster
2    67
0    61
1    50
3    44
4    18
Name: count, dtype: int64

#### 분류모델 돌려보기

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score, cross_validate


In [18]:
# # csv import
# train_df = economics

In [19]:
# 데이터프레임 불러오기
economics_reg = pd.read_csv('./datasets/resampled_data_final.csv',encoding="euc-kr")
economics_reg['DATE'] = pd.to_datetime(economics_reg['DATE'])
economics_reg['DATE'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 289 entries, 0 to 288
Series name: DATE
Non-Null Count  Dtype         
--------------  -----         
287 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.4 KB


In [20]:
economics_reg.drop(['장단기10년물국채/3개월금리차','신규실업수당청구','금융상태지수레버리지','제조업신규주문'],axis=1,inplace=True)

In [21]:
economics_reg = economics_reg.iloc[:287 , ]

In [22]:
# economics_reg.loc[economics_reg['DATE'].dt.year >=2020 ]

In [23]:
economics_reg

,DATE,장단기10년물국채/2년물금리차,국제유가,실업률,신규주택착공건수,주택가격지수,금융상태지수,소비자감성지수,기준금리,M2통화량,경기동행종합,산업생산지수,소매판매
0,2000-01-31,0.07,27.259,4.0,1636.0,100.000,-0.193,112.0,5.45,4666.2,86.22,91.4251,268044.0
1,2000-02-29,-0.11,29.366,4.1,1737.0,100.571,-0.204,111.3,5.73,4679.4,86.42,91.7357,272020.0
2,2000-03-31,-0.47,29.842,4.0,1604.0,101.466,-0.130,107.1,5.85,4710.2,86.85,92.0933,275192.0
3,2000-04-30,-0.45,25.722,3.8,1626.0,102.540,-0.044,109.2,6.02,4766.1,87.19,92.6838,271046.0
4,2000-05-31,-0.40,28.788,4.0,1575.0,103.702,0.021,110.7,6.27,4753.9,87.38,92.9376,271394.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,2023-07-31,-0.91,76.070,3.5,1451.0,310.146,-0.286,71.5,5.12,20863.7,138.67,103.1900,694415.0
283,2023-08-31,-0.76,81.386,3.8,1305.0,311.519,-0.340,69.4,5.33,20825.5,138.80,103.2411,699540.0
284,2023-09-30,-0.44,89.425,3.8,1356.0,312.417,-0.358,67.9,5.33,20755.3,139.21,103.3346,705304.0
285,2023-10-31,-0.19,85.640,3.8,1376.0,312.953,-0.336,63.8,5.33,20725.6,139.38,102.4592,703528.0


In [24]:
# 주택가격지수. 제조업신규주문 , 경기동행종합지수는 차분하고 (변동치로 변경한다는 뜻)
# 제조업 신규주문은 앞으로 어떻게 처리할지 일단 보류
#M2와 소매판매는 변화율로 바꾸는 전처리 시행
economics_reg.주택가격지수 = economics_reg.주택가격지수.diff().fillna(method= 'bfill')
economics_reg.경기동행종합 = economics_reg.경기동행종합.diff().fillna(method= 'bfill')
economics_reg['M2차분'] = economics_reg.M2통화량.diff().fillna(method= 'bfill')
economics_reg['M2변화율'] = economics_reg.M2차분/ (economics_reg.M2통화량.shift(1)) * 100
#economics_reg['M2변화율'] .fillna(method = 'ffill')
economics_reg['소매판매차분'] = economics_reg.소매판매.diff().fillna(method='bfill')
economics_reg['소매판매변화율'] = economics_reg.소매판매차분/ (economics_reg.소매판매.shift(1)) * 100


# 차분, 변화율 확인
economics_reg[['M2통화량','M2차분','M2변화율','소매판매','소매판매차분','소매판매변화율']].head(10)
# 결측치를 다음날 데이터로 채워주기
economics_reg['M2변화율'] = economics_reg['M2변화율'].fillna(method = 'bfill')
economics_reg['소매판매변화율'] = economics_reg['소매판매변화율'].fillna(method = 'bfill')
# M2, 소매판매 원데이터를 지웠습니다.
economics_reg.drop(['M2통화량','M2차분','소매판매','소매판매차분'],axis = 1,inplace = True)


In [25]:
past_data = economics_reg.loc[economics_reg['DATE'].dt.year <2020 ]
present_data = economics_reg.loc[economics_reg['DATE'].dt.year >=2020 ]

In [26]:
# 다시 날짜를 인덱스로 설정해주기
economics_reg = economics_reg.set_index('DATE')

In [27]:
# past_data.reset_index(inplace=True)
# present_data.reset_index(inplace=True)

In [28]:
past_data.set_index('DATE', inplace=True)
past_data

,장단기10년물국채/2년물금리차,국제유가,실업률,신규주택착공건수,주택가격지수,금융상태지수,소비자감성지수,기준금리,경기동행종합,산업생산지수,M2변화율,소매판매변화율
DATE,,,,,,,,,,,,
2000-01-31,0.07,27.259,4.0,1636.0,0.571,-0.193,112.0,5.45,0.20,91.4251,0.282885,1.483339
2000-02-29,-0.11,29.366,4.1,1737.0,0.571,-0.204,111.3,5.73,0.20,91.7357,0.282885,1.483339
2000-03-31,-0.47,29.842,4.0,1604.0,0.895,-0.130,107.1,5.85,0.43,92.0933,0.658204,1.166091
2000-04-30,-0.45,25.722,3.8,1626.0,1.074,-0.044,109.2,6.02,0.34,92.6838,1.186786,-1.506584
2000-05-31,-0.40,28.788,4.0,1575.0,1.162,0.021,110.7,6.27,0.19,92.9376,-0.255974,0.128391
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-31,0.00,54.806,3.6,1367.0,0.359,-0.538,89.8,2.13,0.24,102.7754,0.498632,0.669311
2019-09-30,0.05,56.947,3.5,1300.0,0.178,-0.505,93.2,2.04,0.39,102.5315,0.626902,-0.554691
2019-10-31,0.17,53.963,3.6,1332.0,0.093,-0.510,95.5,1.83,0.21,101.6022,0.884189,0.271944


In [29]:
# past_data 민멕스 스케일링
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(past_data)

# 데이터 변환
past_scaled_data = mMscaler.transform(past_data)

# 데이터 프레임으로 저장
past_scaled_data = pd.DataFrame(data = past_scaled_data, index = past_data.index , columns = past_data.columns)

In [30]:
present_data.set_index('DATE',inplace=True)

In [31]:
# past_data 민멕스 스케일링
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 선언 및 Fitting
mMscaler = MinMaxScaler()
mMscaler.fit(present_data)

# 데이터 변환
present_scaled_data = mMscaler.transform(present_data)

# 데이터 프레임으로 저장
present_scaled_data = pd.DataFrame(data = present_scaled_data, index = present_data.index , columns = present_data.columns)

In [32]:
# past_data['cluster'] = scaled_pca['gmm_cluster']

In [33]:
past_scaled_data.head()

,장단기10년물국채/2년물금리차,국제유가,실업률,신규주택착공건수,주택가격지수,금융상태지수,소비자감성지수,기준금리,경기동행종합,산업생산지수,M2변화율,소매판매변화율
DATE,,,,,,,,,,,,
2000-01-31,0.163142,0.068739,0.076923,0.645125,0.627085,0.175077,1.000000,0.831530,0.703704,0.346574,0.279060,0.506405
2000-02-29,0.108761,0.087142,0.092308,0.701393,0.627085,0.172010,0.987654,0.874807,0.703704,0.362563,0.279060,0.506405
2000-03-31,0.000000,0.091300,0.076923,0.627298,0.676664,0.192640,0.913580,0.893354,0.916667,0.380972,0.420244,0.476300
2000-04-30,0.006042,0.055314,0.046154,0.639554,0.704055,0.216616,0.950617,0.919629,0.833333,0.411371,0.619081,0.222681
2000-05-31,0.021148,0.082094,0.076923,0.611142,0.717521,0.234737,0.977072,0.958269,0.694444,0.424436,0.076356,0.377830


In [34]:
# past_scaled_data에 cluster 컬럼 추가
past_scaled_data.reset_index('DATE',inplace=True)
past_scaled_data['cluster'] = scaled_pca['gmm_cluster']
past_scaled_data

,DATE,장단기10년물국채/2년물금리차,국제유가,실업률,신규주택착공건수,주택가격지수,금융상태지수,소비자감성지수,기준금리,경기동행종합,산업생산지수,M2변화율,소매판매변화율,cluster
0,2000-01-31,0.163142,0.068739,0.076923,0.645125,0.627085,0.175077,1.000000,0.831530,0.703704,0.346574,0.279060,0.506405,3
1,2000-02-29,0.108761,0.087142,0.092308,0.701393,0.627085,0.172010,0.987654,0.874807,0.703704,0.362563,0.279060,0.506405,3
2,2000-03-31,0.000000,0.091300,0.076923,0.627298,0.676664,0.192640,0.913580,0.893354,0.916667,0.380972,0.420244,0.476300,3
3,2000-04-30,0.006042,0.055314,0.046154,0.639554,0.704055,0.216616,0.950617,0.919629,0.833333,0.411371,0.619081,0.222681,3
4,2000-05-31,0.021148,0.082094,0.076923,0.611142,0.717521,0.234737,0.977072,0.958269,0.694444,0.424436,0.076356,0.377830,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2019-08-31,0.141994,0.309343,0.015385,0.495265,0.594644,0.078896,0.608466,0.318393,0.740741,0.930879,0.360217,0.429159,0
236,2019-09-30,0.157100,0.328043,0.000000,0.457939,0.566947,0.088096,0.668430,0.304482,0.879630,0.918323,0.408469,0.313010,0
237,2019-10-31,0.193353,0.301980,0.015385,0.475766,0.553940,0.086702,0.708995,0.272025,0.712963,0.870483,0.505253,0.391452,0
238,2019-11-30,0.193353,0.328934,0.015385,0.483565,0.560826,0.073599,0.731922,0.228748,0.759259,0.898581,0.424573,0.434743,0


In [35]:
past_scaled_data.set_index('DATE', inplace=True)

In [36]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# 결정 트리 기반의 Regressor 생성
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg2 = DecisionTreeRegressor(max_depth=2)
rf_reg7 = DecisionTreeRegressor(max_depth=7, random_state=0)

# 실제 예측을 적용할 테스트용 데이터셋
X_test = present_scaled_data

# 보스턴 주택가격 데이터에서 시각화를 위해 피처는 RM만, 그리고 결정 데이터인 MEDV 추출
features = past_scaled_data.drop(labels = 'cluster', axis=1)
label = past_scaled_data['cluster']

# 학습과 예측 수행
rf_reg.fit(features, label)
rf_reg2.fit(features, label)
rf_reg7.fit(features, label)

pred_rf = rf_reg.predict(X_test)
pred_rf2 = rf_reg2.predict(X_test)
pred_rf7 = rf_reg7.predict(X_test)

In [37]:
pred_rf

array([0.57, 1.7 , 1.91, 3.02, 3.27, 3.16, 1.99, 1.84, 1.65, 1.8 , 0.98,
       0.94, 1.43, 1.82, 1.04, 0.97, 1.29, 1.22, 1.28, 1.95, 1.82, 1.69,
       1.69, 1.7 , 1.69, 2.26, 2.96, 3.08, 3.3 , 3.6 , 3.64, 3.71, 3.71,
       3.71, 3.71, 3.64, 3.58, 3.56, 3.44, 3.4 , 3.44, 3.57, 3.55, 3.56,
       3.46, 3.58, 3.2 ])

In [38]:
pred_rf2

array([0.64705882, 2.51851852, 3.71428571, 3.71428571, 3.71428571,
       3.71428571, 2.51851852, 2.51851852, 2.51851852, 2.51851852,
       1.95555556, 0.64705882, 1.95555556, 1.95555556, 0.64705882,
       0.64705882, 0.64705882, 0.64705882, 0.64705882, 1.95555556,
       1.95555556, 1.95555556, 1.95555556, 1.95555556, 1.95555556,
       2.51851852, 3.71428571, 3.71428571, 3.71428571, 3.71428571,
       3.71428571, 3.71428571, 3.71428571, 3.71428571, 3.71428571,
       3.71428571, 3.71428571, 3.71428571, 3.71428571, 3.71428571,
       3.71428571, 3.71428571, 3.71428571, 3.71428571, 3.71428571,
       3.71428571, 2.51851852])

In [39]:
pred_rf7

array([1., 1., 4., 2., 4., 4., 3., 3., 1., 1., 1., 1., 2., 2., 1., 1., 1.,
       1., 1., 2., 2., 2., 2., 2., 2., 3., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 3.])

In [40]:
present_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 47 entries, 2020-01-31 to 2023-11-30
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   장단기10년물국채/2년물금리차  47 non-null     float64
 1   국제유가              47 non-null     float64
 2   실업률               47 non-null     float64
 3   신규주택착공건수          47 non-null     float64
 4   주택가격지수            47 non-null     float64
 5   금융상태지수            47 non-null     float64
 6   소비자감성지수           47 non-null     float64
 7   기준금리              47 non-null     float64
 8   경기동행종합            47 non-null     float64
 9   산업생산지수            47 non-null     float64
 10  M2변화율             47 non-null     float64
 11  소매판매변화율           47 non-null     float64
dtypes: float64(12)
memory usage: 4.8 KB


In [41]:
present_data['cluster'] = pred_rf7
# present_data.to_csv('국면예측_2020_2023.csv')

In [42]:
present_data

,장단기10년물국채/2년물금리차,국제유가,실업률,신규주택착공건수,주택가격지수,금융상태지수,소비자감성지수,기준금리,경기동행종합,산업생산지수,M2변화율,소매판매변화율,cluster
DATE,,,,,,,,,,,,,
2020-01-31,0.18,57.519,3.6,1572.0,0.161,-0.589,99.8,1.55,0.41,101.3768,0.491492,-0.078796,1.0
2020-02-29,0.27,50.543,3.5,1565.0,0.820,-0.484,101.0,1.58,0.37,101.6330,0.352689,0.070096,1.0
2020-03-31,0.47,29.208,4.4,1267.0,1.976,0.140,89.1,0.65,-0.89,97.6672,3.419998,-8.995753,4.0
2020-04-30,0.44,16.548,14.8,925.0,2.045,0.344,71.8,0.05,-14.04,84.5979,6.376614,-14.622913,2.0
2020-05-31,0.49,28.563,13.2,1054.0,1.248,-0.074,72.3,0.05,1.87,85.9732,5.021297,19.011874,4.0
2020-06-30,0.50,38.307,11.0,1266.0,1.327,-0.334,78.1,0.08,1.91,91.5625,1.571892,8.577351,4.0
2020-07-31,0.44,40.710,10.2,1529.0,1.758,-0.433,72.5,0.09,0.29,95.0144,0.852650,1.589697,3.0
2020-08-31,0.58,42.339,8.4,1377.0,2.481,-0.505,74.1,0.10,1.47,95.8881,0.326474,0.998238,3.0
2020-09-30,0.56,39.634,7.8,1463.0,2.753,-0.497,80.4,0.09,0.75,95.8444,1.260765,1.991341,1.0
